In [ ]:
import sys
from pathlib import Path

home = Path.home()
base = home/'Documents/GitHub/yolov5'
sys.path.append(str(base))

from utils.yolo_utils.general import *
import utils.yolo_utils.imgaug_util as iu
import utils.yolo_utils.bbox_util as bu
import utils.yolo_utils.file_util as fu
import utils.yolo_utils.plot_util as pu

In [ ]:
dpath = Path('/Users/lohcy/Documents/AI_stuffs/datasets/temp/rpc_640_sample')
tpath = dpath/'train'
tl_path = tpath/'labels'
ti_path = tpath/'images'

vpath = dpath/'val'
vl_path = vpath/'labels'
vi_path = vpath/'images'

In [ ]:
bi_path = home/'Documents/AI_stuffs/datasets/coco128/images/train2017'
fi_path = home/'.darwin/datasets/silkronai/lightbox_china_1/images/train'

# PasteProduct

## yaml

In [ ]:
%%writefile ../utils/paste_data.yaml
back_img: /Users/lohcy/Documents/AI_stuffs/datasets/coco128/images/train2017
# back_img: '../test_back_img_paths.txt'
front_img: /Users/lohcy/.darwin/datasets/silkronai/lightbox_china_1/images/train
coco_path: /Users/lohcy/.darwin/datasets/silkronai/lightbox_china_1/releases/clean_coco.json

# img_dest: ./img_dest
# lbl_dest: ./lbl_dest

In [ ]:
%%writefile ../utils/paste_hyp.yaml
#     chance, (min, max)
hsv_h: [0.8, [-4, 4]]   
hsv_s: [0.8, [-20, 80]]
hsv_v: [0.8, [-30, 60]]

rotate: [-360, 360]   # (min, max)
overlap_thereshold: 0.75  # thereshold to remove overlap bbox
depth_min_size: 0.3   # minimum size depth level resize
batch_resize: [0.4, 0.8]  # (min, max)
class_aug: 0.6  # chance for class level optional augmentation
motion_blur: [0.7, [5,20], [0, 360]]  #chance, kernel_size, angle

max_paste_n: 10   # maximum num of small images to paste in background
paste_dist: [4, 4.5, 3.5, 3, 1, 1, 1, 1, 1, 1]  # weighted paste_n distribution
                  



## code

In [ ]:
%autoreload

from utils.paste_product import PasteProduct
from utils.paste_product import create_paste_instance

In [ ]:
paste_data = base/'utils'/'paste_data.yaml'
paste_hyp = base/'utils'/'paste_hyp.yaml'

pp = create_paste_instance(paste_data, paste_hyp, cache='none')

In [ ]:
pp.show_next_batch(5)

# load datas
- pasted_imgs 
    - train
    - val
    
- back_img_path
- front_img_path

In [ ]:
gen_path = home/'Documents/AI_stuffs/datasets/generated'

In [ ]:
len(pp.back_fnames)

In [ ]:
# train
train_imgs, train_bboxes = pp.generate(6)

# val
pp.set_back_fnames(str(vi_path))
val_imgs, val_bboxes = pp.generate(6)

pu.show_batch(train_imgs)
pu.show_batch(val_imgs)

In [ ]:
'''save generated images'''
train_path = gen_path/'train'
val_path = gen_path/'val'

# train
pp.set_back_fnames(bi_path)
pp.gen_then_save(6, train_path/'images', train_path/'labels')

# val
pp.set_back_fnames(vi_path)
pp.gen_then_save(6, val_path/'images', val_path/'labels')

In [ ]:
train_img_paths = pd.Series((train_path/'images').ls())
n = 6

print('train_imgs:')
pu.show_batch(
    [imageio.imread(p) for p in train_img_paths.sample(n)]
)

In [ ]:
val_img_paths = pd.Series((val_path/'images').ls())
n = 6

print('val_imgs:')
pu.show_batch(
    [imageio.imread(p) for p in val_img_paths.sample(n)]
)